### **Crawl**

In [ ]:
from urllib.request import urlopen, Request
import json

data = []

for i in range(1, 17):
    url = 'https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=' + str(i) + '&id=3&order=view_count2&dir=desc&fearture=flashsale_samsung'

    try:
        rep = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        page = urlopen(rep, timeout=30).read()
        crawl_data = json.loads(page)
        data.extend(crawl_data)

    except:
        print(url + ': không tồn tại hoặc bị ẩn')
        #f_temp.write(url+'\n')



In [ ]:
data

In [ ]:
data[1]

In [ ]:
with open('Data\Cellphones\Cellphones_raw_data.json', "w", encoding="utf-8") as outfile:
        json.dump(data, outfile)

### **Tranform to csv**

In [ ]:
import json

with open('Data\Cellphones\Cellphones_raw_data.json', "r", encoding="utf-8") as infile:
        data = json.load(infile)

len(data)

In [ ]:
data[5]

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(data)
df

In [ ]:
cols_drop = ['isLastPage', 'laptop_ram', 'hdd_sdd', 'laptop_cpu', 'tinh_trang_may_cu', 'linh_kien_attribute']

df.drop(columns=cols_drop, inplace=True)

In [ ]:
df['price'] = pd.to_numeric(df['price'], downcast='integer')
df['final_price'] = pd.to_numeric(df['final_price'], downcast='integer')
df['entity_id'] = pd.to_numeric(df['entity_id'])
df['name'] = df['name'].astype('string')

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(df.query("count_all_reviews != count_all_reviews")[['name', 'price', 'final_price', 'stock_available','count_all_reviews', 'average_ratings', 'url']])

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(df.query("price == 0")[['name', 'price', 'final_price', 'stock_available','count_all_reviews', 'average_ratings', 'url']])

In [ ]:
df.drop(df[df.price == 0].index, inplace=True)
df[['stock_available', 'count_all_reviews']] = df[['stock_available', 'count_all_reviews']].fillna(0)

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(df.query("average_ratings != average_ratings")[['name', 'price', 'final_price', 'stock_available','count_all_reviews', 'average_ratings', 'url']])

In [ ]:
df['average_ratings'] = df['average_ratings'].fillna(0.0)
df['promotion_information'] = df['promotion_information'].fillna('')

In [ ]:
len(df)

In [ ]:
df.loc[3, ['promotion_information']]

In [ ]:
# delete html tags in promotion_information
# pip install beautifulsoup4
# pip install lxml

from bs4 import BeautifulSoup

IDs = list(df.index)

for i in range(len(IDs)):
    df.loc[IDs[i], ['promotion_information']] = BeautifulSoup(df['promotion_information'].iloc[i], "lxml").text

In [ ]:
len(df['promotion_information'])

In [ ]:
df.info()

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(df.promotion_information)

In [ ]:
df.to_csv (r'Data\Cellphones\Cellphones_data.csv', index=False, header=True)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('Data\Cellphones\Cellphones_data.csv')

### **Crawl specification**

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

item_list = []

for i in range(len(df)):

    url = df['url'].iloc[i]
    rep = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    page = urlopen(rep).read()
    page = page.decode("utf-8")

    soup = BeautifulSoup(page, "html.parser")
    tables_specification = soup.findAll('table', class_='')

    specifications = {}
    specifications['url'] = url.split('/')[-1]

    for table in tables_specification[1:]:
        for tag in table.findAll('tr', class_=''):
            specification = BeautifulSoup(str(tag), 'lxml').text.strip().split('\n')
            try:
                specifications[specification[0]] = specification[1]
            except:
                print(specifications)
                specifications[specification[0]] = np.nan

    item_list.append(specifications)


In [ ]:
pd.DataFrame.from_dict(item_list).to_csv (r'Data\Cellphones\Cellphones_specification_data.csv', index=False, header=True)